In [172]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [207]:
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import itertools
from sklearn.preprocessing import StandardScaler

In [208]:
import sys
sys.path.append('/Users/Tim/PycharmProjects/HOI/')

In [209]:
from HOI.preprocessings import compute_kernel, compute_kernel_n
from HOI.tests import test_independence
from synthetic_data import stationary_pb_ts, nonstationary_ts_n, stationary_pb_ts_n

In [210]:
os.getcwd()

'/Users/tim/PycharmProjects/HOI/examples'

# data preprocessing from Felix's code

In [211]:
df_VA = pd.read_excel('/Users/tim/PycharmProjects/HOI/examples/wgidataset.xlsx', sheet_name='VoiceandAccountability', header=13).rename(columns={'Unnamed: 0': 'Country'})
df_PS = pd.read_excel('/Users/tim/PycharmProjects/HOI/examples/wgidataset.xlsx', sheet_name='Political StabilityNoViolence', header=13).rename(columns={'Unnamed: 0': 'Country'})
df_GE = pd.read_excel('/Users/tim/PycharmProjects/HOI/examples/wgidataset.xlsx', sheet_name='GovernmentEffectiveness', header=13).rename(columns={'Unnamed: 0': 'Country'})
df_RQ = pd.read_excel('/Users/tim/PycharmProjects/HOI/examples/wgidataset.xlsx', sheet_name='RegulatoryQuality', header=13).rename(columns={'Unnamed: 0': 'Country'})
df_RL = pd.read_excel('/Users/tim/PycharmProjects/HOI/examples/wgidataset.xlsx', sheet_name='RuleofLaw', header=13).rename(columns={'Unnamed: 0': 'Country'})
df_CC = pd.read_excel('/Users/tim/PycharmProjects/HOI/examples/wgidataset.xlsx', sheet_name='ControlofCorruption', header=13).rename(columns={'Unnamed: 0': 'Country'})

In [212]:
dfs = [df_VA, df_PS, df_GE, df_RQ, df_RL, df_CC]

In [213]:
for df in dfs:
    df.drop(columns=[c for c in df if c!='Country' and c not in list(np.arange(1996, 2021, 1))],inplace=True)
    df.drop(index=0, inplace=True)
    df.dropna(inplace=True)

In [214]:
columns = ['Country', 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

In [215]:
for df in dfs:
    for y in np.arange(1996, 2021, 1):
        if y not in df.columns:
            df[y] = np.nan
    
    df[np.arange(1996, 2021, 1)] = df[np.arange(1996, 2021, 1)].astype(float).interpolate(axis=1)

In [216]:
df_VA = df_VA[columns]
df_PS = df_PS[columns]
df_GE = df_GE[columns]
df_RQ = df_RQ[columns]
df_RL = df_RL[columns]
df_CC = df_CC[columns]

In [217]:
countries = set(df_VA['Country']) & set(df_PS['Country']) & set(df_GE['Country']) & set(df_RQ['Country']) & set(df_RL['Country']) & set(df_CC['Country'])

In [218]:
df_VA = df_VA.loc[df_VA['Country'].isin(countries)]
df_PS = df_PS.loc[df_PS['Country'].isin(countries)]
df_GE = df_GE.loc[df_GE['Country'].isin(countries)]
df_RQ = df_RQ.loc[df_RQ['Country'].isin(countries)]
df_RL = df_RL.loc[df_RL['Country'].isin(countries)]
df_CC = df_CC.loc[df_CC['Country'].isin(countries)]

In [219]:
df_RQ.shape

(182, 26)

In [220]:
df_VA.head()

,Country,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
2,Andorra,1.563217,1.545988,1.528759,1.532207,1.535655,1.486392,1.437128,1.422262,1.563077,...,1.360453,1.433838,1.406642,1.165939,1.194477,1.176598,1.161647,1.036913,1.110124,1.089068
3,Afghanistan,-1.908540,-1.973921,-2.039301,-2.035359,-2.031417,-1.732419,-1.433421,-1.177571,-1.203034,...,-1.335977,-1.267430,-1.240068,-1.135445,-1.117563,-1.037838,-0.990854,-1.011603,-1.006792,-1.083459
4,Angola,-1.578164,-1.495021,-1.411879,-1.434972,-1.458065,-1.348422,-1.238779,-1.272615,-1.285480,...,-1.125478,-1.076247,-1.113843,-1.145494,-1.177248,-1.139600,-1.097939,-0.906303,-0.783532,-0.810628
6,Albania,-0.648298,-0.517835,-0.387373,-0.336256,-0.285139,-0.146644,-0.008150,0.070308,0.007238,...,0.062334,0.022064,0.048893,0.143761,0.157149,0.170927,0.203785,0.180734,0.142711,0.087340
8,United Arab Emirates,-0.409404,-0.464283,-0.519163,-0.518002,-0.516841,-0.574192,-0.631544,-0.912192,-0.687913,...,-0.896244,-1.000819,-1.019484,-1.061369,-1.112425,-1.050371,-1.096847,-1.126925,-1.140149,-1.184305


In [221]:
countries = list(df_RQ['Country'])

In [222]:
countries[0]

'Andorra'

# dHSIC tests

In [223]:
df_VA.drop('Country', axis=1, inplace = True)
df_PS.drop('Country', axis=1, inplace = True)
df_GE.drop('Country', axis=1, inplace = True)
df_RQ.drop('Country', axis=1, inplace = True)
df_RL.drop('Country', axis=1, inplace = True)
df_CC.drop('Country', axis=1, inplace = True)

In [224]:
va_arr = df_VA.to_numpy()
ps_arr = df_PS.to_numpy()
ge_arr = df_GE.to_numpy()
rq_arr = df_RQ.to_numpy()
rl_arr = df_RL.to_numpy()
cc_arr = df_CC.to_numpy()

In [229]:
va_arr[0,:].shape

(25,)

In [235]:
from statsmodels.tsa.stattools import adfuller
values = cc_arr[0,:]
res = adfuller(values)
print('Augmneted Dickey_fuller Statistic: %f' % res[0])
print('p-value: %f' % res[1])

Augmneted Dickey_fuller Statistic: -2.815458
p-value: 0.056099


In [200]:
va_arr = StandardScaler().fit_transform(va_arr)
ps_arr = StandardScaler().fit_transform(ps_arr)
ge_arr = StandardScaler().fit_transform(ge_arr)
rq_arr = StandardScaler().fit_transform(rq_arr)
rl_arr = StandardScaler().fit_transform(rl_arr)
cc_arr = StandardScaler().fit_transform(cc_arr)

In [201]:
k_va = compute_kernel_n(va_arr)
k_ps = compute_kernel_n(ps_arr)
k_ge = compute_kernel_n(ge_arr)
k_rq = compute_kernel_n(rq_arr)
k_rl = compute_kernel_n(rl_arr)
k_cc = compute_kernel_n(cc_arr)

In [202]:
def find_comb(var):
    power_set = []
    for L in range(len(var) + 1):
        for subset in itertools.combinations(var, L):
            if len(subset) >= 2:
                power_set.append(list(subset))
    return power_set

In [203]:
power_set = find_comb([k_va, k_ps, k_ge, k_rq, k_rl, k_cc])

In [204]:
power_set_name = find_comb(['va', 'ps', 'ge', 'rq', 'rl', 'cc'])

In [205]:
power = {}
for (name, klist) in tqdm(zip(power_set_name, power_set), total=len(power_set)):
    rejects = 0
    for i in tqdm(range(100)):
        _, _, _, reject = test_independence(klist, None, n_perms=1000, alpha=0.05, mode = 'permutation')
        rejects = rejects + reject
    power[str(name)] = rejects
    print(power)

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{"['va', 'ps']": 100}


  0%|          | 0/100 [00:00<?, ?it/s]

{"['va', 'ps']": 100, "['va', 'ge']": 100}


...

{"['va', 'ps']": 100, "['va', 'ge']": 100, "['va', 'rq']": 100, "['va', 'rl']": 100, "['va', 'cc']": 100, "['ps', 'ge']": 100, "['ps', 'rq']": 100, "['ps', 'rl']": 100, "['ps', 'cc']": 100, "['ge', 'rq']": 100, "['ge', 'rl']": 100, "['ge', 'cc']": 100, "['rq', 'rl']": 100, "['rq', 'cc']": 100}


  0%|          | 0/100 [00:00<?, ?it/s]

{"['va', 'ps']": 100, "['va', 'ge']": 100, "['va', 'rq']": 100, "['va', 'rl']": 100, "['va', 'cc']": 100, "['ps', 'ge']": 100, "['ps', 'rq']": 100, "['ps', 'rl']": 100, "['ps', 'cc']": 100, "['ge', 'rq']": 100, "['ge', 'rl']": 100, "['ge', 'cc']": 100, "['rq', 'rl']": 100, "['rq', 'cc']": 100, "['rl', 'cc']": 100}


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [206]:
power.values()

dict_values([100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100])